In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the dataset
data_path = r'credit_scoring_dataset_large.xlsx'
df = pd.read_excel(data_path)

In [3]:
# Preprocess the data
categorical_features = ['payment_history', 'credit_mix', 'job_stability', 'industry_stability', 'utility_bills_payment_history']
numerical_features = ['late_payments', 'bankruptcies', 'credit_utilization', 'outstanding_balances', 'oldest_account_age', 
                      'avg_account_age', 'recent_inquiries', 'employment_income', 'bonus_commission_income', 'outstanding_loans', 
                      'credit_card_balances']

In [4]:
# Encode categorical features
encoder = OneHotEncoder(sparse_output=False)  # Updated parameter
encoded_features = encoder.fit_transform(df[categorical_features])

# Scale numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numerical_features])

In [5]:
# Combine encoded and scaled features
X = np.hstack((encoded_features, scaled_features))
y = df['credit_score']


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Define the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))


In [8]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [9]:
# Train the model
model.fit(X_train, y_train, epochs=550, batch_size=32, validation_split=0.2)

Epoch 1/550
6/6 [==============================] - 2s 31ms/step - loss: 34550.0977 - val_loss: 31683.0703
Epoch 2/550
6/6 [==============================] - 0s 7ms/step - loss: 34330.0977 - val_loss: 31532.5000
Epoch 3/550
6/6 [==============================] - 0s 7ms/step - loss: 34159.9023 - val_loss: 31365.8828
Epoch 4/550
6/6 [==============================] - 0s 6ms/step - loss: 33952.2344 - val_loss: 31143.6152
Epoch 5/550
6/6 [==============================] - 0s 7ms/step - loss: 33660.3711 - val_loss: 30794.3691
Epoch 6/550
6/6 [==============================] - 0s 6ms/step - loss: 33191.5508 - val_loss: 30221.2285
Epoch 7/550
6/6 [==============================] - 0s 6ms/step - loss: 32456.9941 - val_loss: 29343.7051
Epoch 8/550
6/6 [==============================] - 0s 6ms/step - loss: 31319.6719 - val_loss: 28051.5234
Epoch 9/550
6/6 [==============================] - 0s 6ms/step - loss: 29699.8770 - val_loss: 26151.3516
Epoch 10/550
6/6 [==============================] - 0s

In [10]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Model Loss: {loss}')

2/2 [==============================] - 0s 7ms/step - loss: 593.8217
Model Loss: 593.8217163085938


In [11]:
import os

# Define the directory and file path
directory = r'C:\Users\aniru\Machine Learning\Humanize'
file_path = os.path.join(directory, 'credit_score_model2.h5')

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the model
model.save(file_path)


In [12]:
# Define the neural expert system function
def neural_expert_system(input_data):
    # Convert input_data to DataFrame to match the original dataset
    input_df = pd.DataFrame([input_data])

    # Preprocess the input data similarly to the training data
    encoded_input = encoder.transform(input_df[categorical_features])
    scaled_input = scaler.transform(input_df[numerical_features])
    combined_input = np.hstack((encoded_input, scaled_input))
    
    # Predict the credit score using the neural network model
    predicted_score = model.predict(combined_input)
    return predicted_score[0][0]

# Load the model for future predictions
def load_neural_expert_system_model():
    global model, encoder, scaler
    model = load_model('credit_score_model2.h5')

# Use the loaded model
load_neural_expert_system_model()

# Example input data
input_data = {
    'payment_history': 'good',
    'late_payments': 1,
    'bankruptcies': 0,
    'credit_utilization': 20,
    'outstanding_balances': 1500,
    'oldest_account_age': 6,
    'avg_account_age': 5,
    'credit_mix': 'moderate',
    'recent_inquiries': 2,
    'employment_income': 60000,
    'bonus_commission_income': 5000,
    'job_stability': 'stable',
    'industry_stability': 'stable',
    'outstanding_loans': 10000,
    'credit_card_balances': 2000,
    'utility_bills_payment_history': 'good'
}

# Get the predicted score from the neural expert system
predicted_score = neural_expert_system(input_data)
print(f'Predicted Credit Score: {predicted_score}')

1/1 [==============================] - 0s 70ms/step
Predicted Credit Score: 345.01983642578125


In [13]:
import pickle

# Save the encoder
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
